In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
pd.set_option('display.max_colwidth', 10)
pd.set_option('display.max_columns', None)

In [3]:
cdf_applicant = pd.read_csv('data/cdf_applicant.csv', index_col=['ApplicantID'])
cdf_applicant_experience = pd.read_csv('data/cdf_applicant_experience.csv', index_col=['ApplicantID'])
cdf_stage = pd.read_csv('data/cdf_stage.csv', index_col=['StageID'])
cdf_pipeline = pd.read_csv('data/cdf_pipeline.csv', index_col=['PipelineID'])

In [4]:
df_merged = pd.merge(
    cdf_applicant, cdf_applicant_experience, on=['ApplicantID']
)

df_merged = pd.merge(
    df_merged, cdf_pipeline, on=['ApplicantID']
)

df_merged = pd.merge(
    df_merged, cdf_stage, on=['StageID']
)

df_merged.drop(columns=['StageID'], inplace=True)

df_merged.set_index(['ApplicantID'], inplace=True)

# drop testing input
# df_merged.drop(index=13, inplace=True)

In [5]:
df_merged.head(2)

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
3,0,29,c,male,false,single,indonesia,pekerj...,pelupa...,motor,178,4000000,1,1,teleko...,pt ico...,mainte...,sales ...,3666666,1,6,Rejected
10,0,29,c,male,false,single,indonesia,konsis...,terlal...,motor,168,3700000,1,1,event ...,victor...,bertug...,market...,3200000,2,1,Rejected


In [6]:
'''Hal yang terlewat'''

df_merged.JobDescription.fillna('', inplace=True)

In [7]:
df_merged.isna().sum()

DiseaseHistory       0
Age                  0
DriverLicenseType    0
Gender               0
IsUsingGlasses       0
MaritalStatus        0
Nationality          0
Strengthness         0
Weaknesses           0
TypeOfVehicle        0
Height               0
ExpectedSalary       0
CityID               0
ProvinceID           0
Industry             6
CompanyName          0
JobDescription       0
Position             0
Salary               0
YearsOfExperience    0
JobID                0
Label                0
dtype: int64

In [8]:
# tfidf = TfidfVectorizer()
# mtrix = tfidf.fit_transform(df_merged.JobDescription)

# cosim = linear_kernel(mtrix, mtrix)

In [9]:
df_merged.head(2)

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
3,0,29,c,male,false,single,indonesia,pekerj...,pelupa...,motor,178,4000000,1,1,teleko...,pt ico...,mainte...,sales ...,3666666,1,6,Rejected
10,0,29,c,male,false,single,indonesia,konsis...,terlal...,motor,168,3700000,1,1,event ...,victor...,bertug...,market...,3200000,2,1,Rejected


In [10]:
cat = ['DriverLicenseType', 'Gender', 'IsUsingGlasses', 'MaritalStatus', 'Nationality', 'TypeOfVehicle', 'CityID', 'ProvinceID', 'JobID', 'Label']
num = ['Age', 'Height', 'ExpectedSalary', 'Salary', 'YearsOfExperience']
txt = ['DiseaseHistory', 'Strengthness', 'Weaknesses', 'Industry', 'CompanyName', 'JobDescription', 'Position']

for col in cat:
    df_merged[col] = df_merged[col].astype('category').cat.codes

In [11]:
df_merged.head(2)

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
3,0,29,4,2,0,2,2,pekerj...,pelupa...,2,178,4000000,1,1,teleko...,pt ico...,mainte...,sales ...,3666666,1,5,1
10,0,29,4,2,0,2,2,konsis...,terlal...,2,168,3700000,1,1,event ...,victor...,bertug...,market...,3200000,2,0,1


In [12]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

import numpy as np

In [13]:
def get_num():
    return df_merged[num + cat]

def get_txt():
    return df_merged[txt]

In [14]:
transfomer_num = FunctionTransformer(get_num)
transfomer_txt = FunctionTransformer(get_txt)

In [15]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('numeric_features', Pipeline([ ('selector', transfomer_num) ])),
        ('text_features', Pipeline([ ('selector', transfomer_txt), ('vec', TfidfVectorizer(analyzer='word')) ]))
    ])),
    ('clf', RandomForestClassifier())
])

# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}

param_grid = {
    'clf__min_samples_leaf': [3],
}

# Training config
# kfold = StratifiedKFold(n_splits=7)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=None, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)

In [16]:
X_train = df_merged.drop(columns=['Label'])
Y_train = df_merged.Label

In [17]:
rf_model.fit(X_train, Y_train)
rf_best = rf_model.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\febri\miniconda3\envs\eris\Lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
